In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import hvplot.pandas
import panel as pn

In [ ]:
###读取barcode序列
def load_file(file_path):
    sequence=[]
    for title, seq, qual in FastqGeneralIterator(file_path):
        sequence.append(seq)
    return sequence

t0_filepath='F:/南科大/python/week14/t0.fq'
t3_filepath='F:/南科大/python/week14/t3.fq'

t0_seq=load_file(t0_filepath)
t3_seq=load_file(t3_filepath)

In [ ]:
###读取barcode参考文件和codon文件
barcode_ref=pd.read_csv('F:/南科大/python/week14/barcode_reference.csv')
codon_table=pd.read_csv('F:/南科大/python/week14/codon_table.csv',header=None)
barcode_ref.columns=['barcode','position','original','mutant']
codon_table.columns=['codon','single','triple']

In [ ]:
###整理数据

def data_reorganization(seq1,seq2):
    ###对barcode计数
    t0_counts = pd.value_counts(seq1)
    t3_counts = pd.value_counts(seq2)
    t0_counts = pd.DataFrame(t0_counts, columns=['t0_counts'])
    t3_counts = pd.DataFrame(t3_counts, columns=['t3_counts'])
    ###合并信息
    mutation_info = pd.merge(barcode_ref, t0_counts, left_on='barcode', right_index=True)
    mutation_info = pd.merge(mutation_info, t3_counts, left_on='barcode', right_index=True)
    ###计算频率与log2FC
    mutation_info['t0_frequency'] = mutation_info['t0_counts'] / mutation_info['t0_counts'].sum()
    mutation_info['t3_frequency'] = mutation_info['t3_counts'] / mutation_info['t3_counts'].sum()
    mutation_info['log2FC'] = np.log2(mutation_info['t3_frequency'] / mutation_info['t0_frequency'])
    ###重新编号
    mutation_info.index=range(len(mutation_info))
    ###筛选wt
    wt_index = mutation_info[mutation_info['position'] == 'wt'].index
    wt_info = mutation_info[mutation_info['original'] == 'wt']
    ###计算wt中位数
    median_wt_score=wt_info['log2FC'].median()
    ###去除wt数据，并重新编号
    mutation_info = mutation_info.drop(mutation_info.index[wt_index])
    mutation_info.index=range(len(mutation_info))
    ###添加原始氨基酸和突变氨基酸的注释
    mutation_info=pd.merge(mutation_info,codon_table,left_on = 'original',right_on ='codon',how='inner')
    mutation_info.rename(columns={'single':'original_single','triple':'original_triple'},inplace=True)
    mutation_info.drop(columns=['codon'],inplace=True)
    mutation_info=pd.merge(mutation_info,codon_table,left_on = 'mutant',right_on ='codon',how='inner')
    mutation_info.rename(columns={'single':'mutant_single','triple':'mutant_triple'},inplace=True)  
    mutation_info.drop(columns=['codon'],inplace=True)
    ###创建一列包含位置，以及突变情况的信息
    mutation_info['change']=mutation_info['position']+'-'+mutation_info['original_single']+'-'+mutation_info['mutant_single']
    mutation_info['enrichment_score']=mutation_info['log2FC']-median_wt_score
    return mutation_info,wt_info


In [ ]:
mutation_info=data_reorganization(t0_seq,t3_seq)[0]
wt_info=data_reorganization(t0_seq,t3_seq)[1]

In [ ]:
###聚类计算
def Clustering(mutant,wt):
    ###分组
    cluster = mutant.groupby("change").agg({'enrichment_score': 'mean'})
    cluster.rename(columns={'enrichment_score':'average_enrichscore'},inplace=True)
    p_value=[]
    for change in cluster.index:
        mutant_group=mutant[mutant['change']==change]['log2FC']
        wt_group=wt['log2FC']
        ###检验方差齐性
        if stats.levene(mutant_group,wt_group)[1]>0.05:
            t_test=stats.ttest_ind(mutant_group,wt_group,equal_var=True)
        else:
            t_test=stats.ttest_ind(mutant_group,wt_group,equal_var=False)
        p_value.append(t_test[1])
    cluster['p_value']=p_value
    ###绘制火山图的数据准备
    cluster['-log10(pvalue)']=-np.log10(cluster['p_value'])
    cluster['sig'] = 'normal'
    cluster.loc[(cluster.average_enrichscore> 1 )&(cluster.p_value < 0.05),'sig'] = 'up'
    cluster.loc[(cluster.average_enrichscore< -1 )&(cluster.p_value < 0.05),'sig'] = 'down'
    info=pd.merge(mutant,cluster,left_on = 'change',right_index=True)
    info['position']=pd.to_numeric(info['position'])
    info=info.sort_values('position',ascending=True)
    return info,cluster

In [ ]:
new_info=Clustering(mutation_info,wt_info)[0]
cluster=Clustering(mutation_info,wt_info)[1]

In [ ]:
###作图
def visualize(data,group_info):
    position_selection=data['position'].unique().tolist()
    pos=pn.widgets.Select(name='Position', options=position_selection,value=1)
    @pn.depends(pos,watch=True)
    def draw__plot(pos):###每个位置突变的情况，包含count数以及average_enrichmentscore
        mutant_count=data[data['position']==pos]['mutant_single'].value_counts()
        score_df=data[data['position']==pos][['mutant_single','average_enrichscore']]
        count_score=pd.merge(mutant_count,score_df,left_index=True,right_on='mutant_single')
        count_score.rename(columns={'mutant_single_x':'count'},inplace=True)
        count_score.drop('mutant_single_y',axis=1,inplace=True)
        ###以突变为横坐标，count值为纵坐标，颜色深浅代表average_enrichmentscore分数
        count_score_plot=count_score.hvplot.bar(x='mutant_single', y='count',c='average_enrichscore',title='The counts and average_enrichscore of each mutant',colorbar=True,cmap='bwr')
        return count_score_plot
    
    @pn.depends(pos)
    def make_table(pos):###展示每个位置的突变的数据框
        data_select=data[data['position']==pos]
        table = data_select.hvplot.table(title='Data table')
        return table
    heatmap=data.hvplot.heatmap(x='position', y='mutant_single', C='average_enrichscore', colorbar=True,cmap='bwr')###绘制全局的热图
    volcano= group_info.hvplot.scatter(x="average_enrichscore", y="-log10(pvalue)", by='sig',use_index=True,hover_cols='all')###绘制全局的火山图
    figure=pn.Column(pos,draw__plot,make_table,heatmap,volcano)
    return figure

In [ ]:
visualize(new_info,cluster)